# <center>Workflow for on-line GC and HPLC analysis in flow chemistry</center>


---
## Section 0: Imports, Paths, and Logging
---

In this section all the necessary python packages are imported, the path to this notebook and the logger for this notebook is set up.

In [1]:
# Activate autoreload to keep on track with changing modules #
%reload_ext autoreload
%autoreload 2

# Import standard libraries #
import json
import logging
from pathlib import Path

# Import tools for data processing and analysis and DaRUS upload #
from FAIRFlowChemistry.tools import initialize_dataset, reading_raw_data_widget, analyzing_raw_data_widget, DaRUS_upload

In [2]:
# Define paths for loggin output #
root                = Path.cwd()
logging_config_path = root / "FAIRFlowChemistry/tools/logger_config.json"

# Read in logger specs and configurate logger (set name to current notebook) #
with open(logging_config_path) as logging_config_json: logging.config.dictConfig( json.load( logging_config_json ) )
logger = logging.getLogger(__name__)

# Set the level of thid-party logger to avoid dumping too much information #
for logger_ in ['markdown_it', 'h5py', 'numexpr', 'git']: logging.getLogger(logger_).setLevel('WARNING')

In [3]:
from sdRDM import DataModel

In [5]:
DataModel.from_git( 'https://github.com/FAIRChemistry/FAIRFlowChemistry.git'  )

TypeError: 'NoneType' object is not callable

---
## Section 1: Dataset and raw data
---
In this section the dataset as well as the to analyze raw data is choosen

In [5]:
git_path = 'https://github.com/FAIRChemistry/FAIRFlowChemistry.git'
branch   = ''
DaRUS_subjects = ['Agricultural Sciences', 'Arts and Humanities', 'Astronomy and Astrophysics', 'Business and Management', 'Chemistry', 
                  'Computer and Information Science', 'Earth and Environmental Sciences', 'Engineering', 'Law', 'Mathematical Sciences', 
                  'Medicine, Health and Life Sciences', 'Physics', 'Social Sciences', 'Other']

init_data = initialize_dataset()
init_data.write_dataset( subjects = DaRUS_subjects )

TypeError: 'NoneType' object is not callable

In [4]:
## Definition of basic meta data of the project ##

init_data.title.value        = 'Electrocatalytic CO2-reduction on carbon'
init_data.description.value  = 'The aim of this project is to blablabla'
init_data.project.value      = 'Project B07'

# List with authors and their affiliation #
init_data.authors.value      = 'Richard Schömig, Maximilian Schmidt' 
init_data.affiliations.value = 'University of Stuttgart, University of Stuttgart'
init_data.identifier.value   = 'xxx-xxx-xxx-xxx, xxx-xxx-xxx-xxx'
init_data.contact_text.value = 'Richard Schömig, richard@web.de'

init_data.related_publication.value = "test, https://id.loc.gov/authorities/subjects/sh2014001146.html"

init_data.topic_classification.value = "homogeneous catalysis (LCSH), https://id.loc.gov/authorities/subjects/sh2014001146.html"
init_data.keywords.value             = "polymer chemistry (Loterre Chemistry Vocabulary), https://skosmos.loterre.fr/ERC/en/page/?uri=http%3A%2F%2Fdata.loterre.fr%2Fark%3A%2F67375%2FERC-KCSKD4X9-P"

init_data.dataset_text.value = "test"

In [16]:
## Search for dataset and raw data ##

rrdw = reading_raw_data_widget()
rrdw.choose_data(root)

In [17]:
#str(root) = "c:/Users/darouich/OneDrive/Dokumente/"

e_chem = str(root)+'/data/Rohdaten/01_EChem/CAD14-Cu@AB/GSTATIC.DTA'
mfm    = str(root)+'/data/Rohdaten/03_MFM/CAD14-Cu@AB/Bench-2h-GSS_CAD14-Cu@AB_200_50c_24h_truncated.csv'
gc     = [str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/REPORT01.CSV']
calib  = str(root)+'/data/calibration/calibration.json'
correc = str(root)+'/data/correction_factors/correction_factors.json'
farada = str(root)+'/data/faraday_coefficients/faraday_coefficients.json'

rrdw.Echem_files.value = [e_chem]
rrdw.MFM_files.value   = [mfm]
rrdw.GC_files.value    = gc
rrdw.calib_files.value = [calib]
rrdw.correction_files.value = [correc]
rrdw.faraday_files.value    = [farada]

---
## Section 2: Analysis of data
---
In this section the raw data of the above choosen dataset is analyzed (if you change the dataset above, then reexecute this cell)

In [62]:
# Provide a typical retention time dictionary to pre assign retention times 

typical_retention_time = {"Hydrogen": 1.7, "Carbon dioxide": 3.0, "Carbon monoxide": 13.6, 
                          "Methane": 3.6, "Ethene": 6.0, "Ethane": 7.1}

ardw = analyzing_raw_data_widget()
ardw.choose_experiment( dataset = rrdw.dataset, 
                        dataset_path = rrdw.dataset_dropdown.value, 
                        typical_retention_time = typical_retention_time)

ValidationError: 1 validation error for PeakAssigner
experiment
  Input should be a valid dictionary or instance of Experiment [type=model_type, input_value=Experiment(id='experiment...raday_efficiency=None)]), input_type=Experiment]
    For further information visit https://errors.pydantic.dev/2.5/v/model_type

---
## Section 3: Upload of data to DaRUS
---
In this section the above choosen dataset, containing the processed as well as the raw data, is uploaded to DaRUS

In [7]:
# Predefined list of all SFB1333 dataverses. If a new dataverse wants to be appended, just add in the list
sfb1333_dataverse_list = [ "sfb1333-"+group_name for group_name in ["hansen-gross", "grabowski", "sarkar", "tallarek", "peters", "schmitz", "inf", "pluhackova", "giesselmann", "klemm", "schlaich", "traa", "naumann", "lotsch", "vanSlageren", "holm", "fyta", "estes", "laschat", "bruckner", "bill", "ludwigs", "dyballa", "ringenberg", "kaestner", "Buchmeiser", "sottmann"] ]

Darus_upload = DaRUS_upload()

Darus_upload.DaRUS( dataset = rrdw.dataset, 
                    dataset_path = rrdw.dataset_dropdown.value,
                    dataverse_list = sfb1333_dataverse_list )

Darus_upload.api_token_text.value = "4afecd82-c92d-4935-b786-2225af43531e"

TypeError: add_topic_classification() got an unexpected keyword argument 'vocab_url'